In [56]:
import pandas as pd
import numpy as np
from astropy.time import Time
from astroquery.jplhorizons import Horizons
from astropy import units as u
from astropy.coordinates import SkyCoord
import matplotlib.pyplot as plt
import time
from astropy.coordinates import GeocentricMeanEcliptic
from astropy.coordinates import HeliocentricMeanEcliptic
from astropy.coordinates import HeliocentricTrueEcliptic
from astropy.coordinates import GeocentricTrueEcliptic
from astropy.coordinates import BarycentricTrueEcliptic
from astropy.coordinates import BarycentricMeanEcliptic
from astropy import coordinates 

Hiaka2016 = pd.read_csv('haumea_s1hst_2016.txt')
Namaka2016 = pd.read_csv('haumea_s2hst_2016.txt')

mixedDF = pd.merge(Hiaka2016, Namaka2016, how='outer', on='time')

paramsDF = mixedDF

date = mixedDF['time']
dateList =[]
for i in date:
    jd = Time(i,format='jd')
    dateList.append(jd)


Haumea1 = Horizons(id='Haumea',location=None,epochs = dateList)

dist1 = Haumea1.vectors()['range']

hRA1 = Haumea1.ephemerides()['RA']
hDEC1 = Haumea1.ephemerides()['DEC']

deltaRAH = paramsDF['xH']
deltaDECH = paramsDF['yH']
deltaRAN = paramsDF['xN']
deltaDECN = paramsDF['yN']

paramsDF['Date'] = paramsDF['time']
paramsDF['RA-Primary'] = hRA1
paramsDF['DEC-Primary'] = hDEC1

paramsDF['RA-Hiaka'] = deltaRAH/np.cos(hDEC1*np.pi/180)/3600+hRA1
paramsDF['DEC-Hiaka'] = deltaDECH/3600+hDEC1

paramsDF['RA-Namaka'] = deltaRAN/np.cos(hDEC1*np.pi/180)/3600+hRA1
paramsDF['DEC-Namaka'] = deltaDECN/3600+hDEC1

hRA2 = paramsDF['RA-Hiaka']
hDEC2 = paramsDF['DEC-Hiaka']
hRA3 = paramsDF['RA-Namaka']
hDEC3 = paramsDF['DEC-Namaka']


moonHC = SkyCoord(ra=hRA2*u.degree, dec=hDEC2*u.degree, frame='icrs',distance = dist1,unit=(u.degree,u.degree))
moonNC = SkyCoord(ra=hRA3*u.degree, dec=hDEC3*u.degree, frame='icrs',distance = dist1,unit=(u.degree,u.degree))
primC = SkyCoord(ra=hRA1, dec=hDEC1, frame='icrs',distance = dist1,unit=(u.degree,u.degree))

moonHEcl = moonHC.transform_to(HeliocentricMeanEcliptic(equinox='J2000'))
moonNEcl = moonNC.transform_to(HeliocentricMeanEcliptic(equinox='J2000'))
primEcl = primC.transform_to(HeliocentricMeanEcliptic(equinox='J2000'))

Lat1 = primEcl.lat.degree
Lon1 = primEcl.lon.degree

Lat2 = moonHEcl.lat.degree
Lon2 = moonHEcl.lon.degree

Lat3 = moonNEcl.lat.degree
Lon3 = moonNEcl.lon.degree

deltaLatH = Lat2-Lat1
deltaLonH = (Lon2-Lon1)*np.cos(Lat1*u.degree)

deltaLatN = Lat3-Lat1
deltaLonN = (Lon3-Lon1)*np.cos(Lat1*u.degree)

In [68]:
import numpy as np
import astropy
from astropy import units as u
from astropy.coordinates import SkyCoord
import matplotlib.pyplot as plt

def ratodeg(hours, minutes, seconds):
	degrees = 0
	degrees += hours*15.0
	degrees += minutes*0.25
	degrees += seconds*0.004166666667
	return degrees

def dectodeg(degree,minutes,seconds):
	degrees = 0
	degrees += degree
	degrees += minutes/60
	degrees += seconds/3600
	return degrees


draH = hRA2
ra_errH = paramsDF['exH']/3600
ddecH = hDEC2
dec_errH = paramsDF['eyH']/3600

draN = hRA3
ra_errN = paramsDF['exN']/3600
ddecN = hDEC3
dec_errN = paramsDF['eyN']/3600

ra_arrayH = []
dec_arrayH = []
ra_arrayN = []
dec_arrayN = []
c1 = []
c2 = []

for i in range(hRA1.size):
    ra_arrayH.append(hRA1[i] + np.random.normal(draH[i],ra_errH[i],10000))
    dec_arrayH.append(hDEC1[i] + np.random.normal(ddecH[i],dec_errH[i],10000))
    ra_arrayN.append(hRA1[i] + np.random.normal(draN[i],ra_errN[i],10000))
    dec_arrayN.append(hDEC1[i] + np.random.normal(ddecN[i],dec_errN[i],10000))
    
for i in range(hRA1.size):
    c1.append(SkyCoord(ra=ra_arrayH[i]*u.degree, dec=dec_arrayH[i]*u.degree, frame = 'icrs'))
    c2.append(SkyCoord(ra=ra_arrayN[i]*u.degree, dec=dec_arrayN[i]*u.degree, frame = 'icrs'))

ctrans1 = []
ctrans2 = []
              
for i in range(hRA1.size):
    ctrans1.append(c1[i].transform_to('geocentrictrueecliptic'))
    ctrans2.append(c2[i].transform_to('geocentrictrueecliptic'))

#for i in range(hRA1.size):
    #plt.scatter(c1[i].ra.degree,c1[i].dec.degree)
    #plt.scatter(ctrans1[i].lon.degree,ctrans1[i].lat.degree)
    #plt.axes().set_aspect('equal')
    #plt.show()
    
eLatH = []
eLonH = []
eLatN = []
eLonN = []
for i in range(hRA1.size):
    eLatH.append(np.std(ctrans1[i].lat.arcsec))
    eLonH.append(np.std(ctrans1[i].lon.arcsec))
    eLatN.append(np.std(ctrans2[i].lat.arcsec))
    eLonN.append(np.std(ctrans2[i].lon.arcsec))
    
#print(np.nanmean(eLatH),np.nanmean(eLonH),np.nanmean(eLatN),np.nanmean(eLonN))

obsDF = pd.DataFrame()
obsDF['time'] = date
obsDF['Lat-Prim'] = Lat1
obsDF['Lon-Prim'] = Lon1
obsDF['deltaLat_Hiaka'] = deltaLatH*3600
obsDF['deltaLon_Hiaka'] = deltaLonH*3600
obsDF['eLat-Hiaka'] = eLatH
obsDF['eLon-Hiaka'] = eLonH
obsDF['deltaLat_Namaka'] = deltaLatN*3600
obsDF['deltaLon_Namaka'] = deltaLonN*3600
obsDF['eLat-Namaka'] = eLatN
obsDF['eLon-Namaka'] = eLonN

print(obsDF)
obsDF.to_csv('HaumeaObsDF.csv',index=False)
#print(pd.read_csv('HaumeaObsDF.csv'))

           time   Lat-Prim    Lon-Prim  deltaLat_Hiaka  deltaLon_Hiaka  \
0   2453746.525  26.781379  192.177435        0.188057       -0.311988   
1   2453746.554  26.781392  192.177521        0.192433       -0.316116   
2   2454138.287  26.929865  193.231555        0.114120       -0.282764   
3   2454138.304  26.929875  193.231608        0.115088       -0.283683   
4   2454138.351  26.929891  193.231728        0.122997       -0.291449   
5   2454138.368  26.929901  193.231781        0.124638       -0.290949   
6   2454138.418  26.929920  193.231910        0.131490       -0.297360   
7   2454138.435  26.929926  193.231953        0.134803       -0.296045   
8   2454138.484  26.929936  193.232087        0.141046       -0.302907   
9   2454138.501  26.929947  193.232140        0.144173       -0.304195   
10  2454138.551  26.929966  193.232270        0.148770       -0.310281   
11  2454138.567  26.929972  193.232313        0.153975       -0.310488   
12  2454469.653  27.049935  194.125850